<a href="https://colab.research.google.com/github/fukushun1994/RNN-Anime-model/blob/master/RNN_Anime_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境構築


## GPUの設定

ランタイム > ランタイムのタイプを変更 で ハードウェアアクセラレータをGPUに設定



# データの準備


## Google Drive のマウント

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


## データの取得

In [22]:
import zipfile
import numpy as np
from PIL import Image

DATA_DIR = '/content/gdrive/My Drive/YuyushikiData/'
ZIP_FILE_NAMES = ['ep1.zip']

data_list = []
for zip_file_name in ZIP_FILE_NAMES:
  data = []
  with zipfile.ZipFile(DATA_DIR+zip_file_name, 'r') as zp:
    image_count = len(zp.namelist())
    print('open {} ({} images)'.format(zip_file_name, image_count))
    for i, image_name in enumerate(sorted(zp.namelist())):
      print('\r{}/{}'.format(i, image_count), end='')
      with zp.open(image_name, 'r') as fp:
        with Image.open(fp) as img:
          img_vec = np.array(img, dtype=np.int8)
          data.append(img_vec[:,:,:3])
  data_list.append(data)




open ep1.zip (26235 images)
26234/26235

In [15]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G         10G        173M        884K        1.9G         11G
Swap:            0B          0B          0B


## データセットの作成

In [23]:
SEQ_LENGTH = 100
FRAME_STRIDE = 30
sequences = []
for d in data_list:
  for i in range(0, len(d)-SEQ_LENGTH+1, FRAME_STRIDE):
    sequences.append(np.array(d[i:i+SEQ_LENGTH]))

data_array = np.array(sequences)
data_list = []
  
input_size = (None,) + data_array[0].shape[1:]

print('')
print(input_size)


(None, 72, 128, 3)


# モデルの構築


In [0]:
# code from https://github.com/keras-team/keras/blob/master/examples/conv_lstm.py

from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
import numpy as np
import pylab as plt

FILTER_NUM = 32
KERNEL_SIZE = (3, 3)

input_shape = input_size

seq = Sequential()
seq.add(ConvLSTM2D(filters=FILTER_NUM, kernel_size=KERNEL_SIZE,
                   input_shape=input_shape,
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=FILTER_NUM, kernel_size=KERNEL_SIZE,
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=FILTER_NUM, kernel_size=KERNEL_SIZE,
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(ConvLSTM2D(filters=FILTER_NUM, kernel_size=KERNEL_SIZE,
                   padding='same', return_sequences=True))
seq.add(BatchNormalization())

seq.add(Conv3D(filters=3, kernel_size=(3, 3, 3),
               activation='sigmoid',
               padding='same', data_format='channels_last'))
seq.compile(loss='mse', optimizer='adam')

# 学習


In [34]:
seq.fit(data_array[:,:-1], data_array[:,1:], batch_size=10,
        epochs=300, validation_split=0.05)

Train on 828 samples, validate on 44 samples
Epoch 1/300


ResourceExhaustedError: ignored